### Based off RRL_training.ipynb
##### Parallelizes training over multiple time series at once

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
# Define da model

from keras import layers

INPUT_SIZE = 10
LEARNING_RATE = 0.005

input = layers.Input(shape=(INPUT_SIZE,))
x = layers.Dense(10, activation='relu')(input)
x = layers.Dense(1, activation='tanh')(x)
F_curr_model = keras.Model(inputs=input, outputs=x)
F_prev_model = keras.Model(inputs=input, outputs=x)
F_curr_model.summary()

F_curr_model.compile(optimizer=keras.optimizers.Adam(LEARNING_RATE), loss='mse')

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 dense (Dense)               (None, 10)                110       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [3]:
# Following 2 functions take whole arrays, of shape (instances, size)

def gen_price_series(size=10000, k=3, a=0.9, instances=1):
    p_series = np.ndarray((instances, size,))
    b_series = np.ndarray((instances, size,))
    p_series[:, 0] = 0
    b_series[:, 0] = 0

    for i in range(1, size):
        p_series[:, i] =  p_series[:, i-1] + b_series[:, i-1] + k * np.random.normal(size=instances)
        b_series[:, i] = a * b_series[:, i-1] + np.random.normal(size=instances)

    # shape: (instances)
    R = np.max(p_series, axis=1) - np.min(p_series, axis=1)
    z_series = np.exp(p_series / np.repeat(R[:, np.newaxis], size, axis=1))

    return z_series

def calc_price_returns(zt):
    # returns the set rt, with the first element (0) being NaN
    rt = np.ndarray(zt.shape)
    rt[:, 0] = np.nan
    rt[:, 1:] = zt[:, 1:] - zt[:, :-1]
    return rt



# Following 2 functions take slices of the arrays, of shape (instances,)

def calc_return(mu, rt, Ft_curr, Ft_prev, rft=0, delta=0):
    '''Calculates the returns (Rt) on-line.'''

    return mu * (rft + Ft_prev * (rt - rft) - delta * tf.math.abs(Ft_curr - Ft_prev))

def calc_DSR(n, Rt, At_prev, Bt_prev):
    '''Calculates the differential Sharpe ratios (DSR, Dt) on-line.'''

    At_curr = At_prev + n * (Rt - At_prev)
    Bt_curr = Bt_prev + n * (Rt ** 2 - Bt_prev)

    dDt_dRt = (Bt_prev - At_prev * Rt) / ((Bt_prev - At_prev ** 2) ** 1.5)

    return dDt_dRt, At_curr, Bt_curr


from math import prod
# Repeat grades of shape (instances,) to shape (instances,) + grad.shape
def reshape_grad(grad, shape):
    # takes a tf.Tensor grad of shape (instances,) and the target shape (variables)
    instances = grad.shape[0]
    total_elements = prod(shape)
    grad = tf.repeat(grad, int(total_elements / instances))
    grad = tf.reshape(grad, shape)
    
    return grad

In [4]:
# Simulate trading using the outputs of the model
# Takes SLICES of the arrays, of shape (instances,)

def test_performance(zt, Ft):
    # zt: (instances, size)
    # Ft: (instances, size)

    instances = zt.shape[0]
    size = zt.shape[1]
    Ft = np.sign(Ft)

    values = np.ones((instances, size))
    owned = np.zeros((instances,))
    money = np.ones((instances,))
    values[:, 0] = money

    values_ideal = np.ones((instances, size))
    owned_ideal = np.zeros((instances,))
    money_ideal = np.ones((instances,))
    values_ideal[:, 0] = money_ideal

    for t in range(INPUT_SIZE, size - 1):

        # Model Ft

        # buy if Ft 1, owned 0 --> owned 1
        # sell if Ft -1, owned 1 --> owned 0

        # hold if Ft 0, owned 0 or 1
        # hold if Ft 1, owned 1
        # hold if Ft -1, owned 0

        # model
        buy = np.clip(Ft[:, t] * (1 - owned), 0, 1) # 1 if BUY, 0 if not
        sell = np.clip(-Ft[:, t] * owned, 0, 1) # 1 if SELL, 0 if not
        decision = buy - sell # 1 if BUY, -1 if SELL, 0 if HOLD
        owned = np.clip(owned + decision, 0, 1)
        money -= decision * zt[:, t]
        values[:, t] = money + owned * zt[:, t]

        # ideal
        deltas_ideal = np.sign(zt[:, t + 1] - zt[:, t])
        buy_ideal = np.clip(deltas_ideal * (1 - owned_ideal), 0, 1)
        sell_ideal = np.clip(-deltas_ideal * owned_ideal, 0, 1)
        decision_ideal = buy_ideal - sell_ideal
        owned_ideal = np.clip(owned_ideal + decision_ideal, 0, 1)
        money_ideal -= decision_ideal * zt[:, t]
        values_ideal[:, t] = money_ideal + owned_ideal * zt[:, t]

    return (values[-1] / zt[-1], values_ideal[-1] / zt[-1]), (values, values_ideal)


In [5]:
# INPUT_SIZE = 0

# zt = gen_price_series(size=50, instances=3)
# _, val_series = test_performance(zt, np.random.rand(3, 50))
# fig, ax = plt.subplots(3, 1)
# print(val_series[1].shape)
# print(val_series[1][:, -5:])
# for i in range(3):
#     ax[i].plot(zt[i])
#     ax[i].plot(val_series[1][i])

# plt.show()

In [6]:
# Ft = np.array((0, 1, 1, 1, -1, -1, 0, -1))
# owned = np.array((1, 0, 0, 1, 1, 1, 0, 0))
# money = np.zeros(8)
# zt = np.arange(8)

# # [HOLD,  BUY,  BUY, HOLD, SELL, SELL, HOLD, HOLD]
# # [OWND, OWND, OWND, OWND, NONE, NONE, NONE, NONE]

# buy = Ft * (1 - owned) # 1 if BUY, 0 if not
# buy = np.clip(buy, 0, 1)
# sell = -Ft * owned # 1 if SELL, 0 if not
# sell = np.clip(sell, 0, 1)
# decisions = buy - sell

# print("own: ", owned)
# print("buy: ", buy)
# print("sell:", sell)
# owned += decisions
# owned = np.clip(owned, 0, 1)
# print("new: ", owned)
# money -= zt * decisions
# print("mony:", money) 

In [14]:
# GENERATION VARS
import time

MU = 3
N = 0.01
RISKFREE_RETURN = 0
TRANS_COST = 0.1
SERIES_LENGTH = 1000
TRADING_DELAY = 100
K = 3
A = 0.9

EPISODES = 10
INSTANCES = 1

BASELINE_SERIES = gen_price_series(size=SERIES_LENGTH, k=K, a=A, instances=INSTANCES)

for ep in range(EPISODES):

    # generate price series
    zt = gen_price_series(size=SERIES_LENGTH, k=K, a=A, instances=INSTANCES)
    rt = calc_price_returns(zt)
    Ft = np.zeros((INSTANCES, SERIES_LENGTH,))

    # breaks if init at one; MUST INIT ZERO
    At = np.zeros((INSTANCES, SERIES_LENGTH,))
    Bt = np.zeros((INSTANCES, SERIES_LENGTH,))

    SR_series = np.zeros((INSTANCES, SERIES_LENGTH,))
    DSR_series = np.zeros((INSTANCES, SERIES_LENGTH,))
    Rt_series = np.zeros((INSTANCES, SERIES_LENGTH,))
    dD_series = np.ones((INSTANCES, SERIES_LENGTH,))
    autodiff_series = np.ones((INSTANCES, SERIES_LENGTH,))


    # plt.plot(zt)
    # plt.show()


    @tf.function
    def jacobian(tape, F, F_vars):
        print("TRACING JACOBIANS")
        dF_dVar = tape.jacobian(F, F_vars) # shape: (instances, MODEL VAR SHAPE)
        return dF_dVar
    
    for t in range(INPUT_SIZE, SERIES_LENGTH):
        Ta = time.time()
        
        with tf.GradientTape(persistent=True) as tape:
            # prediction, set new position

            F_curr = tf.reshape(F_curr_model(zt[:, t - INPUT_SIZE + 1:t + 1]), (INSTANCES,))
            F_prev = tf.reshape(F_prev_model(zt[:, t - INPUT_SIZE:t]), (INSTANCES,)) # On the first iteration, this does not exist and is not used.
            
            
            Ft[:, t] = F_curr

            # if the first iteration, F(t-1) does not yet exist so no update can be made.
            F_prev_model.set_weights(F_curr_model.get_weights())
            # calculate the gradient
            Rt = calc_return(MU, rt[:, t], F_curr, F_prev)
            dDt_dRt, At[:, t], Bt[:, t] = calc_DSR(N, Rt, At[:, t - 1], Bt[:, t - 1])
            
            Rt_series[:, t] = Rt
            SR_series[:, t] = np.mean(Rt_series[:, INPUT_SIZE:t]) / np.std(Rt_series[:, INPUT_SIZE:t])
            DSR = (Bt[:, t-1] * (Rt - At[:, t-1]) - (1/2) * (At[:, t-1] * (Rt ** 2 - Bt[:, t-1]))) / ((Bt[:, t-1] - At[:, t-1] ** 2) ** 1.5)
            DSR_series[:, t] = (Bt[:, t-1] * (Rt - At[:, t-1]) - (1/2) * (At[:, t-1] * (Rt ** 2 - Bt[:, t-1]))) / ((Bt[:, t-1] - At[:, t-1] ** 2) ** 1.5)
            dD_series[:, t] = dDt_dRt

        # calculate derivatives.
        dRt_dFcurr = tape.gradient(Rt, F_curr) # shape: (instances,)
        dRt_dFprev = tape.gradient(Rt, F_prev) # shape: (instances,)



        # Tgrad = time.time()

        
        # print(F_curr)

        # dFcurr_dThetacurr = tape.jacobian(F_curr, F_curr_model.trainable_variables) # shape: (instances, MODEL VAR SHAPE)
        # dFprev_dThetaprev = tape.jacobian(F_prev, F_prev_model.trainable_variables) # shape: (instances, MODEL VAR SHAPE)
        # print(dFcurr_dThetacurr)
        # dFcurr_dThetacurr = jacobians(F_curr, F_curr_model)
        # dFprev_dThetaprev = jacobians(F_prev, F_prev_model)
        dFcurr_dThetacurr = jacobian(tape, F_curr, F_curr_model.trainable_variables) # shape: (instances, MODEL VAR SHAPE)
        dFprev_dThetaprev = jacobian(tape, F_prev, F_prev_model.trainable_variables) # shape: (instances, MODEL VAR SHAPE)
        
        # print("Time grad:", time.time() - Tgrad)



        autodiff_series[:, t] = tape.gradient(DSR, Rt)

        # print(len(dRt_dFcurr))
        # print(dRt_dFcurr.shape)
        # print(len(dFcurr_dThetacurr))
        # print(dFcurr_dThetacurr[0].shape)

        # dont update parameters if DSR hasn't stabilized yet
        if t < TRADING_DELAY:
            continue

        # Set F(t-1) to F(t) for the next iteration.
        F_prev_model.set_weights(F_curr_model.get_weights())

        # print(len(F_curr_model.trainable_variables))
        
        if t != INPUT_SIZE:
            # multiply derivatives together.
            gradient_update = []


            for i in range(len(dFcurr_dThetacurr)):
                total_elements = prod(dFcurr_dThetacurr[i].shape)
                # dDt_dRt = dDt_dRt.numpy().repeat(total_elements).reshape((INSTANCES,) + dFcurr_dThetacurr[i].shape)
                # dRt_dFcurr = dRt_dFcurr.numpy().repeat(total_elements).reshape((INSTANCES,) + dFcurr_dThetacurr[i].shape)
                # dRt_dFprev = dRt_dFprev.numpy().repeat(total_elements).reshape((INSTANCES,) + dFcurr_dThetacurr[i].shape)

                dDt_dRt_exp = reshape_grad(dDt_dRt, dFcurr_dThetacurr[i].shape)
                dRt_dFcurr_exp = reshape_grad(dRt_dFcurr, dFcurr_dThetacurr[i].shape)
                dRt_dFprev_exp = reshape_grad(dRt_dFprev, dFcurr_dThetacurr[i].shape)

                grad = dDt_dRt_exp * (dRt_dFcurr_exp * dFcurr_dThetacurr[i] + dRt_dFprev_exp * dFprev_dThetaprev[i])
                grad = tf.reduce_sum(grad, axis=0)
                grad *= LEARNING_RATE

                # gradient_update.append(tf.reshape(grad, F_curr_model.trainable_variables[i].shape))
                gradient_update.append(grad)


            VERBOSE = False
            if VERBOSE:
                X = -1
                print("\nSAAAAA========================")
                # a = F_curr_model.trainable_variables[X].numpy()
                # print("bias variable:", F_curr_model.trainable_variables[X])
                # print(F_curr_model.trainable_variables[-2])
                # print("grad update:", gradient_update[X])

                # print("grad update:", gradient_update[X].numpy())
                # print("grad update:", gradient_update)
            # if np.isnan(gradient_update[X].numpy()):
            #     dasufgapg = 1

            vars = F_curr_model.trainable_variables
            for i in range(len(vars)):
                vars[i].assign_add(gradient_update[i])

            if VERBOSE:
                # b = F_curr_model.trainable_variables[X].numpy()
                # print("delta:", b - a)
                # print("new bias :", F_curr_model.trainable_variables[X])
                # print("output:", F_curr)
                print("BASELINE output:", F_curr_model(BASELINE_SERIES[-INPUT_SIZE:].reshape(1, INPUT_SIZE)))

        Tb = time.time()
        print("Time:", Tb - Ta)

    if ep % 1 == 0:
        print("Episode: ", ep)
        # test performance
        deltas, val_series = test_performance(zt, Ft)

        fig, ax = plt.subplots(3, 2, figsize=(15, 6))

        ax[0, 0].plot(zt[0])
        ax[0, 0].plot(val_series[0][0])
        ax[0, 0].set_title("price")

        ax[1, 0].plot(SR_series[0, :])
        ax[1, 0].set_title("SR")

        ax[0, 1].plot(DSR_series[0, :])
        ax[0, 1].set_title("DSR")

        ax[1, 1].plot(dD_series[0, :])
        ax[1, 1].set_title("dD/dR")
        ax[2, 1].plot(autodiff_series[0, :])
        ax[2, 1].set_title("autodiff_series")
        plt.show()


    # plt.plot(val_series[1])


TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS


C:\Users\jason\AppData\Local\Temp\ipykernel_8704\2138195476.py:65: RuntimeWarning: divide by zero encountered in scalar divide
  SR_series[:, t] = np.mean(Rt_series[:, INPUT_SIZE:t]) / np.std(Rt_series[:, INPUT_SIZE:t])


TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS
TRACING JACOBIANS


KeyboardInterrupt: 

: 